In [ ]:
from getfilelistpy import getfilelist
from google_drive_downloader import GoogleDriveDownloader as gdd
from datetime import datetime, timedelta
import os
from google.cloud import storage


def list_file_from_gdrive(request=None):
    resource = {
        "api_key": " ## API Key from GCP APIs & Services >> Credentials ## ",
        "id": " ## Google Drive ID ## ",
        "fields": "files(name,id,createdTime,modifiedTime)",
    }
    res = getfilelist.GetFileList(resource)
    files = res['fileList']
    # Sort modified time descending order
    sorted_files = sorted(files, key=lambda x: x.get('modifiedTime', 0), reverse=True)

    for files in sorted_files:
        for file in files.get('files'):
            modified_time = datetime.fromisoformat(file.get('modifiedTime')[:-1]).replace(tzinfo=None)
            print(modified_time)
            if modified_time >= datetime.now() - timedelta(hours=24):
                download_file(file['id'], file['name'])
    return "Completed"

def download_file(id, name):
    gdd.download_file_from_google_drive(file_id=id, dest_path='/tmp/'+name+'.xlsx', unzip=False)
    upload_to_gcs('/tmp/'+name+'.xlsx', name)
    os.remove('/tmp/'+name+'.xlsx')
    return "Completed"

def upload_to_gcs(path, destination):
    storage_client = storage.Client()
    bucket = storage_client.bucket(' ## Insert your bucket name ## ')
    blob = bucket.blob(destination)
    blob.upload_from_filename(path)

if __name__ == '__main__':
    list_file_from_gdrive()